In [1]:
#  !pip install geemap

Defaulting to user installation because normal site-packages is not writeable


# Machine Learning with Earth Engine - Supervised Classification

### ขั้นตอนการจำแนกประเภทที่ใช้การควบคุม (Supervised classification algorithms) ที่มีให้ใช้ใน Google Earth Engine

Source: https://developers.google.com/earth-engine/classification

แพ็คเกจ Classifier ใน Earth Engine ใช้ในการจำแนกประเภทโดยใช้วิธีการควบคุมของอัลกอริทึมแบบดั้งเดิม อัลกอริทึมที่สามารถใช้ในการจำแนกประเภทรวมถึง CART, RandomForest, NaiveBayes, และ SVM ขั้นตอนการทำงานทั่วไปสำหรับการจำแนกประเภทคือ:

รวบรวมข้อมูลการ training dataset โดยเก็บฟีเจอร์ที่มีคุณสมบัติเก็บป้ายชื่อคลาสที่รู้จักและคุณสมบัติที่เก็บค่าตัวเลขสำหรับตัวทำนายสร้างอ็อบเจ็กต์ของคลาสิฟายเออร์ (classifier) และตั้งค่าพารามิเตอร์ตามที่จำเป็นฝึกคลาสิฟายเออร์โดยใช้ข้อมูลการฝึกอบรมจำแนกภาพหรือคอลเลคชั่นของคุณสมบัติประมาณความผิดพลาดในการจำแนกด้วยข้อมูลทดสอบอิสระข้อมูลการฝึกอบรมเป็น FeatureCollection ที่มีคุณสมบัติเก็บป้ายชื่อคลาสและคุณสมบัติที่เก็บตัวแปรทำนาย ป้ายชื่อคลาสควรเป็นเลขจำนวนเต็มติดต่อกันเริ่มต้นที่ 0 หากจำเป็นต้องใช้ remap() เพื่อแปลงค่าคลาสให้เป็นเลขจำนวนเต็มติดต่อกัน ตัวแปรทำนายควรเป็นตัวเลข

![](https://i.imgur.com/vROsEiq.png)

## บทแนะนำขั้นตอนทีละขั้นตอน

### Import libraries

In [2]:
import ee
import geemap

### สร้างแผนที่โดยใช้ไลบรารี geemap และแสดงผลแผนที่

In [3]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### เพิ่มข้อมูลลงในแผนที่

In [4]:
# กำหนดจุดที่ตั้งข้อมูลแผนที่พื้นที่ศึกษาจาก FAO กรองด้วยการใช้พิกัดภูมิศาสตร์ 
point = ee.Geometry.Point([100.25181617367595, 16.819711283484004])  # อำเภอเมืองพิษณุโลก 

# โหลดข้อมูลสำหรับกรองพื้นที่อำเภอเมืองพิษณุโลกด้วยจุดพิกัด point
Mueang = ee.FeatureCollection('FAO/GAUL/2015/level2').filterBounds(point)
                    
# โหลดข้อมูลภาพถ่ายดาวเทียม Landsat-8 และกรองข้อมูลภาพ
image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-08-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

# ตัดขอบเขตพื้นที่ศึกษาของภาพถ่ายดาวเทียม
image_M = image.clip(Mueang)

# สร้างแผนที่โฟกัสโดยกำหนดพิกัดศูนย์กลางและระดับการซูม
Map.centerObject(Mueang, 8)

# เพิ่มชั้นข้อมูล การผสมสีข้อมูลจากดาวเทียม (Band Combinations) ลงในแผนที่
# ลักษณะของสิ่งปกคลุมดินจะปรากฏคล้ายกับการมองเห็นของมนุษย์ พืชหรือต้นไม้ที่มีสุขภาพดีจะมีสีเขียว ถนนเป็นสีเทา
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'],'min':0, 'max': 3000}, 'ภาพผสมสีธรรมชาติ (Red, Green, Blue)')

# การผสมรูปแบบนี้มีความใกล้เคียงกับการผสมสีธรรมชาติ แตกต่างที่เมืองจะสามารถแยกได้ชัดเจนมากกว่า
Map.addLayer(image, {'bands': ['B7', 'B6', 'B4'],'min':0, 'max': 3000}, 'ความเป็นเมือง (SWIR2, SWIR1, Red)')

# รูปแบบนี้ได้รับความนิยมอย่างมาก เพื่อศึกษาเกี่ยวกับพืชพรรณโดยเฉพาะ เนื่องจากพืชจะปรากฏเป็นสีแดงแยกกับวัตถุชนิดอื่นๆ อย่างชัดเจน
Map.addLayer(image, {'bands': ['B5', 'B4', 'B3'],'min':0, 'max': 3000}, 'สีอินฟราเรด (NIR, Red, Green)') 

# เป็นเทคนิคที่ใช้กันอย่างแพร่หลายในการวิเคราะห์ภาพดาวเทียมเพื่อทำการแยกแยะแหล่งน้ำและพื้นที่บนบก
Map.addLayer(image, {'bands': ['B5', 'B6', 'B4'],'min':0, 'max': 3000}, 'แยกแยะระหว่างพื้นที่บนบกและน้ำ(NIR, SWIR1, Red)') 

# การผสมในรูปแบบนี้มีประโยชน์ในการตรวจสอบพืชผลการเกษตร พืชที่แข็งแรงสมบูรณ์จะแสดงผลสีเขียวสดใส ส่วนที่รกล้างว่างเปล่าจะแสดงเป็นสีน้ำตาลหรือสีม่วง
Map.addLayer(image, {'bands': ['B6', 'B5', 'B2'],'min':0, 'max': 3000}, 'เกษตรกรรม (SWIR1, NIR, Blue)') 

# เพิ่มแผนที่ควบคุมเพื่อให้ผู้ใช้สามารถเปลี่ยนแปลงและเลือกชั้นข้อมูลได้
Map.addLayerControl()

### ตรวจสอบคุณสมบัติของภาพ

In [5]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-12-10'

In [6]:
image.get('CLOUD_COVER').getInfo()

0.02

### สร้างชุดข้อมูลสำหรับจุด training dataset

มีวิธีหลายวิธีที่คุณสามารถสร้างพื้นที่เพื่อสร้างชุดข้อมูลการสุ่มจุด training area ได้

- วาดรูปทรง (เช่น สี่เหลี่ยม) บนแผนที่และใช้ region = Map.user_roi
- กำหนดรูปทรงเรขาคณิต เช่น region = ee.Geometry.Rectangle([100.25181617367595,16.819711283484004,100.25472150345898,16.8008333638423])
- สร้างโซน buffer รอบจุด เช่น region = ee.Geometry.Point([100.25181617367595,16.819711283484004]).buffer(7000)

หากไม่กำหนดพื้นที่ มันจะใช้รอบขอบภาพเป็นค่าเริ่มต้นโดยอัตโนมัติ

In [ ]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

ในตัวอย่างนี้ เราจะใช้ [ESA WorldCover 10m v100] (https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100) เพื่อสร้างชุดข้อมูลป้ายกำกับสำหรับการ training dataset


![](https://data.apps.fao.org/map/wmts/wmts?request=GetLegendGraphic&VERSION=1.0.0&layer=ESA/WorldCover/v100&FORMAT=image/png&WIDTH=20&HEIGHT=20&STRICT=false&sld=https://storage.googleapis.com/gee-wmts-review/catalog/CGIAR/SRTM90_V4/styles/default.xml?ignoreCache=1)

In [7]:
# โหลดข้อมูล Land Cover จาก European Space Agency (ESA)
dataset = ee.ImageCollection('ESA/WorldCover/v100').first().clip(image.geometry())

# กำหนดการแสดงผลในแบนด์ของข้อมูล
visualization = {
    'bands': ['Map'],
}

# สร้างแผนที่ geemap
Map = geemap.Map()

# เพิ่มข้อมูลแผนที่ Landcover ลงในแผนที่
Map.add_layer(dataset, visualization, 'Landcover')

# เพิ่มการควบคุมชั้นข้อมูล
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
# ข้อมูลสำหรับจุด training dataset บนภาพ
points = dataset.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  
    }
)

# เพิ่มชั้นข้อมูลจุด training dataset ลงในแผนที่
Map.addLayer(points, {}, 'training')

In [9]:
# ประมวลผลและพิมพ์จำนวนข้อมูลทั้งหมดในจุด training dataset ที่ถูกสร้างขึ้น 
print(points.size().getInfo())

5000


In [10]:
# พิมพ์ข้อมูลของจุดแรกใน training dataset 
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [100.14835836961247, 17.522350569187868]}, 'id': '0', 'properties': {'Map': 40}}


### Train the classifier 
โดยใช้โมเดลแบบตัวแบ่งประเภทแบบ CART (Classification and Regression Trees) ในการฝึกโมเดล โดยใช้ข้อมูลจากขั้นตอนที่แล้วและกำหนดแถบที่ใช้ในการคาดการณ์ เช่นกันโดยใช้ชุดแถบที่กำหนดไว้ก่อนแล้วเป็นพารามิเตอร์ในการฝึกโมเดล โดยการใช้ข้อมูลที่สุ่มมาเพื่อฝึกโมเดลในการทำนายป้ายชื่อของพื้นที่ปกคลุมดิน

In [11]:
# ใช้พื้นที่ของแถบดังกล่าวสำหรับการคาดการณ์
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# คุณสมบัตินี้ของตารางเก็บป้ายชื่อของพื้นที่ปกคลุมดิน
label = 'Map'

# ซ้อนทับจุดบนภาพเพื่อทำการ training dataset
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# training ตัวแบ่งประเภทแบบ CART ด้วยพารามิเตอร์เริ่มต้น
trained = ee.Classifier.smileCart().train(training, label, bands)

In [12]:
# ใช้เพื่อแสดงข้อมูลของจุดแรกในชุดข้อมูลการฝึกอบรมเมื่อมีการดำเนินการ getInfo() บนข้อมูลนั้น
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 692, 'B2': 784, 'B3': 1062, 'B4': 1384, 'B5': 2353, 'B6': 3757, 'B7': 3231, 'Map': 40}}


### จำแนกภาพ

In [20]:
# จัดกลุ่มพิกเซลในภาพด้วยแบนด์เดียวกันที่ใช้สำหรับการ training 
result = image.select(bands).classify(trained)

landcover = result

# กำหนดสีที่จะใช้แสดงผล
landcover_palette = [
    '#009933', '#ffbb22', '#ffff4c', '#FFFF33','#FFCC00', '#ff0000', '#CC0000',
    '#87cefa', '#6495ed', '#6600ff', '#00cf75','#006400' 
]

# กำหนดรูปแบบและพารามิเตอร์ของแผนที่
Map = geemap.Map(center=[16.812037436738137, 100.30989048991965], zoom=11)

# เพิ่มชั้นข้อมูลแผนที่สำหรับ Landcover 
Map.add_layer(dataset.clip(Mueang), visualization, 'Land cover (ESA)')

# เพิ่มชั้นข้อมูลแผนที่ผลลัพธ์จากการจัดกลุ่มพิกเซล
Map.addLayer(landcover.clip(Mueang), {'min': 10, 'max': 100, 'palette': landcover_palette}, 'Land Cover')

# เพิ่มการควบคุมชั้นข้อมูล
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[16.812037436738137, 100.30989048991965], controls=(WidgetControl(options=['position', 'transparent…

### ฟังก์ชันการแสดงภาพผลลัพธ์

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

### ส่งออกผลการจำแนกประเภทข้อมูลแบบกำกับดูแล (Supervised classification)

ส่งออกผลลัพธ์ไปยังคอมพิวเตอร์โดยตรง:

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
geemap.ee_export_image(result, filename=out_file, scale=90)

ส่งออกผลลัพธ์ไปยัง Google Drive:

In [ ]:
geemap.ee_export_image_to_drive(
    landcover, description='landcover', folder='export', scale=10,maxPixels=1e13        
)